# Native DuckDB vs JupySQL

## Native DuckDB

In [2]:
import duckdb
from pandas import DataFrame
import numpy as np

num_rows = 1_000_000

df = DataFrame(np.random.randn(num_rows, 20))

In [3]:
with duckdb.connect() as con:
    %timeit con.sql("select * from df").df()

85.9 ms ± 682 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## JupySQL

In [4]:
%load_ext sql
%sql duckdb://
%config SqlMagic.autopandas = True

In [6]:
%%timeit
df_ = %sql select * from df

*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://
*  duckdb://